In [1]:
#Usando a biblioteca do Python

%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai
client = genai.Client()
modelo = "gemini-2.5-flash"

In [4]:
# Instalar Framework de agentes do Google

!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [5]:
from google.adk.agents import Agent # Agente
from google.adk.runners import Runner # Orquestrador/Coordenador
from google.adk.sessions import InMemorySessionService # Memória interna do orquestrador
from google.adk.tools import google_search # Ferramenta do Google Search (para poder pesquisar)
from google.genai import types  # Para criar conteúdos (Content e Part) / fazer as configurações
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [6]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [7]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [49]:
####################################################
# --- Agente 1: Referencia Musical do Usuário --- #
####################################################
def agente_referencial (info_usuario):
  referencial = Agent(
      name="agente_referencial",
      model="gemini-2.0-flash",
      description="Agente inteligente que coleta suas preferências musicais habituais e seu humor atual para encontrar playlists perfeitas no YouTube.",
      instruction="""
      Você é um Agente de Recomendação Musical Personalizada. Sua tarefa é interagir com o usuário para entender tanto suas preferências
      musicais habituais quanto seu estado de espírito atual, a fim de encontrar playlists relevantes no YouTube. Primeiramente,
      pergunte sobre os gêneros musicais, artistas ou músicas que o usuário costuma gostar ou ouvir com frequência. Incentive-o
      a compartilhar exemplos e, se possível, mencionar playlists que ele aprecia. Em seguida, pergunte sobre o humor ou sentimento
      que ele está experimentando no momento (por exemplo: feliz, relaxado, agitado, nostálgico, etc.). Com base nas informações
      fornecidas sobre suas preferências musicais e seu estado de espírito, você deverá construir uma linha de raciocínio para
      conectar esses dois aspectos. Use a ferramenta de busca do Google (google_search) com foco no site youtube.com para pesquisar
      artistas, gêneros, playlists e como o humor pode influenciar a preferência musical. Sua resposta final deve apresentar
      uma análise concisa das preferências musicais e do humor do usuário, juntamente com sugestões de artistas, músicas ou
      playlists relevantes que combinem ambos os aspectos, prontas para serem pesquisadas no YouTube.
      """,
      tools=[google_search]
  )
  entrada_do_agente_referencial = f"""
  Informações do usuário:
  Estilo musical desejado: {info_usuario.get('pergunta_1', '')}
  Humor atual: {info_usuario.get('pergunta_2', '')}
  Artistas/bandas que combinam com o humor: {info_usuario.get('pergunta_3', '')}
  Música recente marcante: {info_usuario.get('pergunta_4', '')}
  Vibe musical procurada: {info_usuario.get('pergunta_5', '')}
  """
  resultados = call_agent(referencial, entrada_do_agente_referencial)
  return resultados


In [60]:
##########################################
# --- Agente 2: Buscador de Playlists --- #
##########################################
def agente_buscador(info_usuario, resultados_referencial):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um Agente de Busca de Playlists Musicais no YouTube. Sua tarefa é utilizar a ferramenta de busca do
        Google (google_search) com foco exclusivo no domínio youtube.com para encontrar as playlists mais relevantes
        para o usuário, com base nas informações fornecidas sobre suas preferências musicais e seu estado de espírito, conforme análise do Agente Referencial:


        E nas informações diretas do usuário:
        Estilo musical desejado:
        Humor atual:
        Artistas/bandas que combinam com o humor:
        Música recente marcante:
        Vibe musical procurada:

        Com base nessas informações, procure no YouTube por no máximo 5 playlists que correspondam aos critérios do usuário. Priorize playlists com boa quantidade de músicas (pelo menos 10) e que
        explorem artistas similares ou relacionados, fugindo um pouco do que o usuário já costuma ouvir, mas mantendo a relevância com seu gosto e humor.

        Sua resposta deve ser uma lista clara de no máximo 5 playlists, incluindo o link direto para cada uma delas.
        """,
        description="Encontra as melhores playlists no YouTube, combinando seu gosto musical e humor, com foco em novidades e variedade.",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"""
    Análise do Agente Referencial: {resultados_referencial}
    Estilo musical desejado: {info_usuario.get('pergunta_1', '')}
    Humor atual: {info_usuario.get('pergunta_2', '')}
    Artistas/bandas que combinam com o humor: {info_usuario.get('pergunta_3', '')}
    Música recente marcante: {info_usuario.get('pergunta_4', '')}
    Vibe musical procurada: {info_usuario.get('pergunta_5', '')}
    """

    playlists_encontradas = call_agent(buscador, entrada_do_agente_buscador)
    return playlists_encontradas

In [61]:
##########################################
# --- Agente 3: Revisor de Qualidade --- #
##########################################
def agente_revisor(info_usuario, playlists_encontradas, resultados_referencial):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
        Você é um Editor e Revisor de Conteúdo especializado em música e profundo conhecedor da plataforma YouTube.
        Sua tarefa é avaliar criticamente as playlists fornecidas pelo Agente Buscador, garantindo a máxima compatibilidade
        com o estado emocional ("vibe") e as preferências musicais (gostos pessoais) previamente expressas pelo usuário.

        Com base na análise do Agente Referencial:


        E nas informações diretas do usuário:
        Estilo musical desejado:
        Humor atual:
        Artistas/bandas que combinam com o humor:
        Música recente marcante:
        Vibe musical procurada:

        Playlists encontradas pelo Agente Buscador:


        Analise cada playlist individualmente, considerando os seguintes aspectos: Coerência com o Humor: As músicas e o clima
        geral da playlist refletem o estado de espírito atual do usuário? Há faixas que destoam significativamente da "vibe" indicada?
        Alinhamento com as Preferências: A seleção de artistas e gêneros na playlist corresponde aos gostos musicais habituais do
        usuário? Mesmo explorando artistas relacionados, a playlist mantém uma conexão clara com suas preferências?
        Qualidade Geral: A playlist parece bem curada? As músicas fluem bem juntas? Há alguma repetição excessiva de artistas ou faixas?

        Se todas as playlists atenderem satisfatoriamente aos critérios de compatibilidade e qualidade, sua resposta deve ser concisa:
        "As playlists estão ótimas!". Caso encontre problemas em alguma playlist, seja específico ao apontá-los e sugira melhorias
        concretas. Por exemplo, você pode sugerir a remoção de faixas que não se encaixam no humor, a inclusão de artistas mais alinhados
        às preferências, ou a busca por playlists com uma curadoria mais apurada.
        """,
        description="Garante que as playlists recomendadas no YouTube correspondam perfeitamente ao seu humor e gosto musical, oferecendo apenas o melhor.",
        tools=[google_search]
    )

    entrada_do_agente_revisor = f"""
    Análise do Agente Referencial: {resultados_referencial}
    Estilo musical desejado: {info_usuario.get('pergunta_1', '')}
    Humor atual: {info_usuario.get('pergunta_2', '')}
    Artistas/bandas que combinam com o humor: {info_usuario.get('pergunta_3', '')}
    Música recente marcante: {info_usuario.get('pergunta_4', '')}
    Vibe musical procurada: {info_usuario.get('pergunta_5', '')}
    Playlists encontradas: {playlists_encontradas}
    """

    playlists_revisadas = call_agent(revisor, entrada_do_agente_revisor)
    return playlists_revisadas

In [11]:
print("Não sabe que musicas ouvir hoje? Vem que esses 3 agentes vão lhe ajudar! 🎶 😊 ")
print("\n")
print("Primeiramente: precisamos conhecer um pouco sobre você e o que voce gosta de ouvir!")
print("\n")

# --- Obter as Referências Musicais e Humor do Usuário ---
info_usuario = {}
cont = 0
num_perguntas = 5

perguntas_usuario = [
    "Qual estilo ou gênero musical você está com mais vontade de ouvir agora?",
    "Como você está se sentindo neste momento? (ex: feliz, relaxado, agitado, nostálgico...)",
    "Pensando em artistas ou bandas que você gosta, algum deles combina com o seu humor de hoje?",
    "Tem alguma música que você ouviu recentemente que te marcou ou que reflete como você está se sentindo?",
    "Além dos seus gostos habituais, existe algum tipo de 'vibe' musical específica que você procura para hoje?"
]

# Criando um chat bot curto para o primeiro agente ter as informações que precisa
while cont < num_perguntas:
    resposta = input(f"{perguntas_usuario[cont]} ")
    info_usuario[f"pergunta_{cont+1}"] = resposta
    cont += 1

print("\nObrigado pelas informações!")
print("Suas respostas:", info_usuario)
print("\n")

# --- Próximos passos (envio para os agentes) ---

Não sabe que musicas ouvir hoje? Vem que esses 3 agentes vão lhe ajudar! 🎶 😊 


Primeiramente: precisamos conhecer um pouco sobre você e o que voce gosta de ouvir!


Qual estilo ou gênero musical você está com mais vontade de ouvir agora? talvez um pop, funk, eletronico.... quero algo animado hoje
Como você está se sentindo neste momento? (ex: feliz, relaxado, agitado, nostálgico...) to feliz, animada, mas não agitada demais
Pensando em artistas ou bandas que você gosta, algum deles combina com o seu humor de hoje? lady gaga, rihanna, ariana grande
Tem alguma música que você ouviu recentemente que te marcou ou que reflete como você está se sentindo? vanish into you da lady gaga
Além dos seus gostos habituais, existe algum tipo de 'vibe' musical específica que você procura para hoje? não sei, me surpreenda. quero ouvir coisas novas

Obrigado pelas informações!
Suas respostas: {'pergunta_1': 'talvez um pop, funk, eletronico.... quero algo animado hoje', 'pergunta_2': 'to feliz, animada, 

In [62]:
# --- Próximos passos (envio para os agentes) ---

# --- Executar os agentes ---

print(f"Maravilha! Vamos agora descobrir suas playlists!")


resultados_referencial = agente_referencial(info_usuario)
print("\n--- 📝 Resultado do Agente 1 (Referencial) ---\n")
display(to_markdown(resultados_referencial))
print("--------------------------------------------------------------")


playlists_encontradas = agente_buscador(info_usuario, resultados_referencial)
print("\n--- 📝 Resultado do Agente 2 (Buscador) ---\n")
display(to_markdown(playlists_encontradas))
print("--------------------------------------------------------------")


post_final = agente_revisor(info_usuario, playlists_encontradas, resultados_referencial)
print("\n--- 📝 Resultado do Agente 3 (Revisor) ---\n")
display(to_markdown(post_final))
print("--------------------------------------------------------------")

Maravilha! Vamos agora descobrir suas playlists!

--- 📝 Resultado do Agente 1 (Referencial) ---



> Olá! Que bom que você está se sentindo feliz e animada! Adoro a Lady Gaga, Rihanna e Ariana Grande, ótimas escolhas para um dia como hoje. "Vanish Into You" é um hino!
> 
> Para te surpreender com algo novo, vou usar suas preferências (pop, funk, eletrônico animado e Lady Gaga, Rihanna, Ariana Grande) e seu estado de espírito (feliz, animada, mas não agitada demais) para encontrar algumas playlists no YouTube que você possa gostar.
> 
> Vou pesquisar no YouTube por playlists que combinem esses elementos. Aqui estão algumas opções que acredito que você vai gostar:
> 
> 
> Com base nas suas preferências e humor, encontrei algumas opções que podem te agradar no YouTube:
> 
> *   **Playlists com hits de Lady Gaga, Rihanna e Ariana Grande:** Remixes e mashups que misturam os maiores sucessos dessas artistas.
> *   **Playlists de Pop/Eletrônico:** Busque por playlists com "pop animado" ou "eletrônico dançante" para descobrir novas músicas e artistas dentro desses gêneros.
> *   **Playlist com vibe positiva:** Uma playlist com músicas alto astral para melhorar o seu dia.
> *   **Músicas para levantar o astral:** Para incentivar, trabalhar, estudar e ser feliz.
> 
> Espero que essas sugestões te ajudem a encontrar algo novo e animado para ouvir!


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Buscador) ---



> Com certeza! Para te surpreender e te apresentar coisas novas, vou buscar no YouTube playlists que combinem pop, funk, eletrônico animado, com a vibe da Lady Gaga, Rihanna e Ariana Grande, mas que também te mostrem artistas e músicas diferentes.
> 
> 
> Com base nas suas preferências e no seu estado de espírito, preparei uma lista de playlists no YouTube que podem te surpreender e te animar:
> 
> 1.  **Pop Funk 2025 | As Mais Animadas Para Dançar:** Uma playlist com os hits mais animados de Pop Funk para você dançar e se divertir.
>     *   [https://www.youtube.com/playlist?list=PLB3LPm8qQ70S-YTvG85G8LPK3K9-q7育](https://www.youtube.com/playlist?list=PLB3LPm8qQ70S-YTvG85G8LPK3K9-q7%E8%82%B2)
> 2.  **FUNK COM ELETRÔNICA:** Essa playlist mistura o ritmo contagiante do funk com a batida envolvente da música eletrônica, criando uma experiência sonora única e perfeita para dançar.
>     *   [https://www.youtube.com/playlist?list=PLw6oYMyDP8HRzxSjF8kWF3VSmF9GvCqR](https://www.youtube.com/playlist?list=PLw6oYMyDP8HRzxSjF8kWF3VSmF9GvCqR)
> 3.  **MÚSICA ELETRÔNICA ANIMADA:** Uma seleção de músicas eletrônicas animadas com vocais e letras positivas, perfeitas para dançar, relaxar, estudar, malhar ou correr.
>     *   [https://www.youtube.com/playlist?list=PLXfSnKj6d-nlh5N-QIx4DzEgSu5mE-vCr](https://www.youtube.com/playlist?list=PLXfSnKj6d-nlh5N-QIx4DzEgSu5mE-vCr)
> 4.  **Playlists de Músicas Alto-Astral - Vibe Positiva | Filtr Music Brasil:** Essa playlist da Filtr Music Brasil traz mais de duas horas de músicas "good vibes" e alto astral para melhorar o seu dia.
>     *   [https://www.youtube.com/watch?v=ODzzVwKtZpw](https://www.youtube.com/watch?v=ODzzVwKtZpw)
> 5.  **Ariana Grande Mashups:** Playlist com 41 videos com Mashups da Ariana Grande.
>     *   [https://www.youtube.com/playlist?list=PLoK4bK0Jp_0baBWJlu9yCjQclEEVDXJml](https://www.youtube.com/playlist?list=PLoK4bK0Jp_0baBWJlu9yCjQclEEVDXJml)
> 
> Espero que você curta bastante essas playlists e descubra novas músicas e artistas que te inspirem e te façam sentir ainda mais feliz e animada!
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Revisor) ---



> As playlists estão ótimas! A combinação de pop, funk e eletrônico animado, juntamente com a inclusão de mashups da Ariana Grande, atende bem ao seu gosto musical e ao seu estado de espírito. As playlists de "Música Eletrônica Animada" e "Vibe Positiva" também parecem adequadas para manter o clima animado, mas não agitado demais, que você busca.


--------------------------------------------------------------
